In [10]:
import requests
from newspaper import Article
import heapq
import re
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from collections import deque
import os
import json
import openpyxl
import pandas as pd
import concurrent.futures
from PyPDF2 import PdfReader
from openpyxl.utils import get_column_letter
import time
from openai import OpenAI
import tiktoken

In [11]:
def extract_text(web_url):
    try:
        if web_url.lower().endswith('.pdf'):
            pdf_filename = os.path.basename(urlparse(web_url).path)

            if not os.path.exists('documents'):
                os.makedirs('documents')

            pdf_filepath = os.path.join('documents', pdf_filename)

            response = requests.get(web_url)
            with open(pdf_filepath, 'wb') as f:
                f.write(response.content)

            try:
                with open(pdf_filepath, 'rb') as f:
                    reader = PdfReader(f)
                    text = ""
                    for page in reader.pages:
                        page_text = page.extract_text()
                        if page_text:
                            text += page_text
                print(f'Done for {web_url}')
               
                return text
            except Exception as e:
                print(f"Failed to read PDF {web_url}: {e}")
                return None  # Return None if PDF reading fails
        else:
            response = requests.get(web_url)
            soup = BeautifulSoup(response.content, 'html.parser')
            text = soup.get_text()
            print(f'Done for {web_url}')
            return text

    except Exception as e:
        print(f'Failed at {web_url} {str(e)}')
        return None 


In [13]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote_plus

query = "latest news on S&P500"
encoded = quote_plus(query)  

# 2) Bing News search URL (not the general web search)
url = f"https://www.bing.com/news/search?q={encoded}"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, "html.parser")

# 3) Grab the news‐card links
for card in soup.select("a.title"):
    title = card.get_text(strip=True)
    link  = card["href"]
    print(f"{link}\n")
    article = Article(link)
    article.download()
    article.parse()
    text = article.text.strip()
    cleaned_text = re.sub(r' {3,}', '  ', text)
    if len(cleaned_text)<50:
        continue
    print("this is it",cleaned_text)


https://www.msn.com/en-in/money/markets/wall-street-live-s-p-500-dow-jones-nasdaq-rise-ahead-of-big-earnings/ar-AA1J07Mh?ocid=BingNewsVerp

https://www.msn.com/en-in/news/other/s-p-500-and-nasdaq-notch-record-high-closes-lifted-by-alphabet/ar-AA1J21yM?ocid=BingNewsVerp

https://www.msn.com/en-us/money/savingandinvesting/s-p-500-and-nasdaq-set-new-record-highs-here-s-why-wall-street-expects-rally-could-climb-further/ar-AA1J17fY?ocid=BingNewsVerp

https://www.moneycontrol.com/news/business/markets/s-p-500-ends-down-as-tariffs-sour-sentiment-13270546.html

this is it International Markets

Wall Street ended lower on Friday, with Meta Platforms weighing on the S&P 500 after President Donald Trump intensified his tariff offensive against Canada, amplifying the uncertainty swirling around U.S. trade policy.

Trump late on Thursday ramped up his tariff assault on Canada, saying the U.S. would impose a 35% tariff on imports next month and planned to impose blanket tariffs of 15% or 20% on most

In [ ]:
YOUR_API_KEY = "pplx-dx"